In [1]:
import numpy as np
import networkx as nx
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression
import matplotlib.pyplot as plt
import matplotlib
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.graphics.correlation import plot_corr
from sklearn.linear_model import LinearRegression
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

EDA

In [ ]:
sales = pd.read_csv("data/Burget_Sales.csv").drop(columns=["Date"])

In [ ]:
X = sales
y = X.pop("burger_sales")

In [ ]:
# bimodal distribution
# no particularly "fat" tails
# burger sales can be negative
fig, ax = plt.subplots(1,1)
sns.kdeplot(y,ax=ax)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

In [ ]:
pacf = plot_pacf(y, method="ywm")

In [ ]:
px.imshow(sales.corr(), color_continuous_scale="Blues")

In [ ]:
sns.pairplot(sales)

In [ ]:
M = sales.shape[1] - 1 # number of media channels

Modelling

In [ ]:
def geometric_decay(l:np.array, alpha:float=0.8) -> np.array:
    return (np.ones(len(l)) * alpha) ** l

In [ ]:
def geometric_delayed_decay(l:np.array, theta_m:int, alpha:float=0.5) -> np.array:
    return (np.ones(len(l)) * alpha) ** ((l - theta_m)**2)

In [ ]:
def adstock(t:int, channel_data:np.array, w_m, L:int, alpha_m:float) -> float:
    l = np.arange(0,L)
    num = np.sum(w_m(l, alpha_m)*channel_data[t-L:t])
    den = np.sum(w_m(l, alpha_m))

    return num/den

In [ ]:
matplotlib.rcParams['mathtext.fontset'] = "cm"
fig, ax = plt.subplots(1,1)
x = np.linspace(0,20, 200)
ax.plot(x, geometric_decay(x))
ax.plot(x, geometric_delayed_decay(x, alpha=0.8, theta_m=5))
ax.legend([plt.Line2D([],[],ls='-', lw=1.75), plt.Line2D([],[],ls='-', lw=1.75)],
                  ['Geometric Addstock', "Geom. Del Addstock"],  loc=(.55, .7))

ax.set_xlabel('$lag$', fontsize = 15)
ax.set_ylabel('$adstock$', fontsize = 15)
ax.set_xlim(x.min(), x.max())

In [ ]:
adstock(20, sales.Google_Cost, geometric_decay, 10, 0.8)

In [ ]:
# Shape effect

# S_m shape paramter, also: slope.
# K_m saturation point, Hill(K_m) = 0.5 for any K_m or S_m

def Hill(x_m:np.array, S_m, K_m):
    return 1 / (1 + (x_m/K_m)**-S_m)


def beta_Hill(x_m:np.array, S_m:float, beta_m:float, K_m:float) -> np.array:
    return beta_m - ((K_m**S_m*beta_m)/(x_m**S_m + K_m**S_m))

In [ ]:
# TODO
# define different functional forms for the hill functiona and compare them.
# (Hill function has poor identifiability)
# - reach transformation
# - sigmoid function
# - integral of other (e.g. normal) distributions

fig, ax = plt.subplots(1,1)
ax.plot(x, beta_Hill(x, S_m=1, beta_m=0.3, K_m=0.5))
ax.plot(x, beta_Hill(x, S_m=2, beta_m=0.8, K_m=1.5))
ax.set_xlabel('Media Spent', fontsize = 15)
ax.set_ylabel('$Hill(x_{m,t})$', fontsize = 15)

In [ ]:
# y_t ...  Sales
# tau ... Baseline sales
